# <center> Блендинг </center>

* Что такое блендинг?
* Что можно смешивать?
* Принципы блендинг?

In [128]:
!mkdir test

In [133]:
mkdir notebooks subs models data tmp_data src

total 24
drwxr-xr-x 2 jovyan root 4096 Jan 27 04:10 data
drwxr-xr-x 2 jovyan root 4096 Jan 27 04:10 models
drwxr-xr-x 2 jovyan root 4096 Jan 27 04:10 notebooks
drwxr-xr-x 2 jovyan root 4096 Jan 27 04:10 src
drwxr-xr-x 2 jovyan root 4096 Jan 27 04:10 subs
drwxr-xr-x 2 jovyan root 4096 Jan 27 04:10 tmp_data


# Принципы блендинга

* 🦑 Не бленди пока не выжал максимум из моделей по отдельности
* 🦐 Чем раньше проверишь эффект от блендинга, тем эффективне будет стратегия
* 🐠 Блендить можно с рзаными весами пропорционально скорам
* 🐬 Против блендинга только больший блендинг
* 🐋 Блендинг по фолдам или чекпоинтам это тоже блендинг
* 🐡 Блендинг по сидам это стабилизирующий блендинг
* 🐙 При равной точности, ансабль побеждает соло-модель на привате 
* 🐳 Чем различнее и сильнее модели, тем эффективнее блендинг
* 🦈 Стекинг сильнее блендинга
* 🎣 Стекать можно с фичами 
* 🎏 Стекинг бывает разных уровней 
* 🐉 Не можешь больше стакать - блендись с решением сокомандников

In [58]:
# path = 'https://stepik.org/media/attachments/lesson/779915/fs_task1_dataset_10ftrs.csv'
# pd.read_csv(path)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,target
0,4187,46433,34.205,0.82,2009,0.829,2013,6,2016,5.24,2896.011206
1,2108,133416,-3.498,0.08,2000,2.121,2017,-18,2018,4.04,789.255097
2,2720,110790,-25.179,-1.32,2005,-0.482,2016,-18,2018,3.00,3963.677442
3,3179,48318,-40.497,-1.21,2015,-1.297,2013,19,2020,4.70,6514.237113
4,3198,41827,12.841,-0.79,2004,-0.472,2013,-8,2022,2.80,7259.402025
...,...,...,...,...,...,...,...,...,...,...,...
147895,2197,78650,-18.281,1.51,2012,-1.537,2015,2,2015,3.10,1005.085904
147896,3020,132752,6.584,-0.97,2015,1.048,2017,11,2020,3.80,1736.313378
147897,4283,43200,-3.506,0.82,2001,0.216,2013,2,2017,3.08,1345.981700
147898,1255,43359,-29.596,0.05,2000,0.999,2013,-7,2021,3.50,7849.148604


In [8]:
!ls ../subs/

rforest_kfold.csv      simple_baseline.csv     Solution.csv
sample_submission.csv  simple_baseline_v2.csv


# Task generation

# Soft voting

In [122]:
import pandas as pd
import numpy as np
solution = pd.read_csv('../subs/Solution.csv').sample(frac=1)
solution['car_id'] = [f'car_{i}' for i in list(range(solution.shape[0]))]
solution['true_target_class'] = solution['target_class']


targets = solution['target_class'].unique()

def make_noize(answer):
    if np.random.random() > 0.47:
        return np.random.choice(targets)
    return answer


    
for num in range(1, 6):
    solution[f'model_{num}'] = solution['true_target_class'].apply(make_noize)
    print(f'model {num}', np.mean(solution['true_target_class'] == solution[f'model_{num}']))
    

def make_blend(row):
    cols = [f'model_{num}' for num in range(1, 6)]
    return row[cols].value_counts().index[0]
    
solution['target_class'] = solution.apply(make_blend, axis=1)


print(np.mean(solution['true_target_class'] == solution['target_class']))

solution[['car_id', 'target_class']].to_csv('../do_not_commit_this/blend_baseline.csv', index=False)
solution['target_class'] = None
print(np.mean(solution['true_target_class'] == solution['target_class']))

# https://stepik.org/media/attachments/lesson/779915/fs_10_task.csv

solution.drop(['true_target_class'], axis=1).to_csv('../do_not_commit_this/make_soft_blend.csv', index=False)

solution.drop(['target_class'], axis=1).rename(
    columns = {'true_target_class' : 'target_class'})[['car_id', 'target_class']].to_csv('../do_not_commit_this/soft_blend_sol.csv', index=False)

solution[['car_id', 'target_class']].to_csv('../do_not_commit_this/sample_submission.csv', index=False)




solution.head()

model 1 0.5211709357030841
model 2 0.5342394145321484
model 3 0.5206481965499216
model 4 0.534762153685311
model 5 0.5279665446941976
0.7914270778881338
0.0


,car_id,target_class,true_target_class,model_1,model_2,model_3,model_4,model_5
42,car_0,None,engine_fuel,engine_ignition,engine_fuel,engine_fuel,gear_stick,engine_fuel
359,car_1,None,wheel_shake,wheel_shake,wheel_shake,wheel_shake,wheel_shake,wheel_shake
784,car_2,None,another_bug,engine_check,another_bug,another_bug,another_bug,electro_bug
1051,car_3,None,break_bug,break_bug,break_bug,engine_fuel,engine_ignition,engine_ignition
49,car_4,None,electro_bug,electro_bug,engine_check,break_bug,engine_fuel,engine_ignition


In [125]:
solution.drop(['target_class'], axis=1).rename(
    columns = {'true_target_class' : 'target_class'}).head()

,car_id,target_class,model_1,model_2,model_3,model_4,model_5
42,car_0,engine_fuel,engine_ignition,engine_fuel,engine_fuel,gear_stick,engine_fuel
359,car_1,wheel_shake,wheel_shake,wheel_shake,wheel_shake,wheel_shake,wheel_shake
784,car_2,another_bug,engine_check,another_bug,another_bug,another_bug,electro_bug
1051,car_3,break_bug,break_bug,break_bug,engine_fuel,engine_ignition,engine_ignition
49,car_4,electro_bug,electro_bug,engine_check,break_bug,engine_fuel,engine_ignition


In [72]:
# solution.drop(['true_target_class'], axis=1).head()

In [106]:
# solution = pd.read_csv('../subs/Solution.csv').sample(frac=1)
# correct_str = "https://stepik.org/media/attachments/lesson/825510/make_soft_blend.csv"
# correct_str = 'https://stepik.org/media/attachments/lesson/825510/soft_blend_solution.csv'
# pd.read_csv(correct_str)

In [120]:
import io
import pandas as pd


reply = '../do_not_commit_this/blend_baseline.csv'

# '../do_not_commit_this/soft_blend_solution.csv' # 
correct_str = "https://stepik.org/media/attachments/lesson/825510/soft_blend_solution.csv"
correct_df = pd.read_csv(correct_str, index_col='car_id')

if reply == "https://stepik.org/media/attachments/lesson/825510/soft_blend_solution.csv":
    student_df = pd.read_csv(reply, index_col='car_id')
    
else:
    # student = io.StringIO(reply)
    student = reply
    student_df = pd.read_csv(student, index_col='car_id')

    try:
        if len(correct_df) != len(student_df):
            print( False, 'Кажется, число строк не совпадает. Сверьтесь с sample_submission.csv')

        if len(correct_df.columns) != len(student_df.columns):
            print( False, 'Кажется, число столбцов не совпадает. Сверьтесь с sample_submission.csv')

        correct_df.sort_index() == student_df.sort_index()

    except:
        print( False, 'Кажется, проблема с форматированием. Сверьтесь с sample_submission.csv')


result = float((correct_df.sort_index() == student_df.sort_index())['target_class'].mean())

threshold = 0.55

if result < threshold:
    print( 0, F'Accuracy вашего решения {result}. Нужно как минимум {threshold}')

answer = (result - threshold) / (1 - threshold)
print(answer, F'Вы побили порог в {threshold} accuracy: ваше решение дает accuracy={result}. Если у вас еще остались попытки, попробуйте улучшить результат!')






False Кажется, число столбцов не совпадает. Сверьтесь с sample_submission.csv
False Кажется, проблема с форматированием. Сверьтесь с sample_submission.csv


ValueError: Can only compare identically-labeled DataFrame objects

In [121]:
correct_df.sort_index()
# student_df.sort_index()

,target_class,model_1,model_2,model_3,model_4,model_5
car_id,,,,,,
car_0,another_bug,engine_ignition,electro_bug,another_bug,another_bug,gear_stick
car_1,electro_bug,electro_bug,electro_bug,electro_bug,break_bug,electro_bug
car_10,gear_stick,gear_stick,gear_stick,engine_check,engine_fuel,gear_stick
car_100,engine_fuel,engine_fuel,engine_check,engine_fuel,engine_fuel,engine_fuel
car_1000,engine_fuel,gear_stick,another_bug,another_bug,engine_fuel,engine_ignition
...,...,...,...,...,...,...
car_995,break_bug,engine_check,gear_stick,electro_bug,break_bug,wheel_shake
car_996,engine_fuel,engine_ignition,gear_stick,engine_fuel,engine_ignition,engine_fuel
car_997,wheel_shake,wheel_shake,break_bug,electro_bug,gear_stick,wheel_shake
